# Downloading data from NCEI, cropping, and coarsening


Resources:

 - [Catalog of some datasets](https://data.noaa.gov/waf/NOAA/NESDIS/NGDC/MGG/DEM/iso/)
 - [NCEI 1/9" tiles](https://data.noaa.gov//metaview/page?xml=NOAA/NESDIS/NGDC/MGG/DEM/iso/xml/999919.xml&view=getDataView&header=none)
 

In [ ]:
%matplotlib inline

In [ ]:
from pylab import *

In [ ]:
from clawpack.geoclaw import topotools

## Download topo from NCEI thredds server

### For etopo1 data use:

In [ ]:
etopo1_server = 'https://www.ngdc.noaa.gov/thredds/dodsC/global/ETOPO1_Ice_g_gmt4.nc'

We can crop and coarsen in the process, so that a much smaller file is downloaded.

Here we crop to a local region and coarsen by a factor 2 to create an array with 2 arc-minute resolution.

In [ ]:
extent = [-125,-122,47,49]
coarsen = 2
etopo1_subset = topotools.read_netcdf(etopo1_server, extent=extent, 
                                      coarsen=coarsen, verbose=True)

In [ ]:
etopo1_subset.plot()
title('topography')
savefig('junk.png')

## For coastal inundation DEMs use:

In [ ]:
server = 'https://www.ngdc.noaa.gov/thredds/dodsC/regional/'

Append to this the name of the netCDF file to access, e.g.

 - 'astoria_13_mhw_2012.nc'
 - 'puget_sound_13_mhw_2014.nc'
 - 'port_townsend_13_mhw_2011.nc'
 - 'strait_of_juan_de_fuca_13_navd88_2015.nc'

Again we can crop and coarsen in the process, so that a much smaller file is downloaded.

Here we crop to a smaller extent and coarsen by a factor 6 to create an array with 2 arc-second resolution.

In [ ]:
path = server + 'puget_sound_13_mhw_2014.nc'
extent = [-123, -122.3, 47.85, 48.04]
PS_subset_2sec = topotools.read_netcdf(path, extent=extent, coarsen=6, verbose=True)

### Check that this agrees with version from local file:

In [ ]:
if 1:
    path = '/Users/rjl/topo/WA/puget_sound_13_mhw_2014.nc'
    extent = [-123, -122.3, 47.85, 48.04]
    PS_subset_2sec_local = topotools.read_netcdf(path, extent=extent, coarsen=6, verbose=True)
    print('Difference in two Z arrays: ',\
          abs(PS_subset_2sec_local.Z - PS_subset_2sec.Z).max())

In [ ]:
PS_subset_2sec.plot()

## Plotting the topography

The topography object (e.g. `PS_subset_2sec` above) has a function `plot()` defined that makes a basic plot with a predetermined colormap as done above, often useful as a first pass.

You can make more fancy and customized plots with any Python plotting commands. 
The `X, Y, Z` attributes of the topography object (e.g. `PS_subset_2sec`) are numpy arrays of the respective variables.

In [ ]:
topo = PS_subset_2sec
topo.extent

In [ ]:
topo2 = topo.crop(filter_region=[-122.6,-122.4,47.9,47.95], coarsen=2)

In [ ]:
topo2.plot()

In [ ]:
topo2.write('topo2.tt3', topo_type=3)

## Grid registration

It is important to understand how the DEM is registered.  When reading a netCDF file from the NCEI thredds server, the latitude and longitude arrays returned give the points where the DEM elevations `Z` are located.  With earlier `.asc` files the the header often specified the lower left corner of a "grid cell" and the DEM values were located at cell centers.  This has caused many headaches.  

See http://www.clawpack.org/grid_registration.html for more discussion, and also:

 - http://www.clawpack.org/topo.html
 - [ESRI page](http://resources.esri.com/help/9.3/arcgisengine/java/GP_ToolRef/spatial_analyst_tools/esri_ascii_raster_format.htm)
 - [NOAA page](https://www.ngdc.noaa.gov/mgg/global/gridregistration.html)

In [ ]:
print('X = \n',PS_subset_2sec.X[:2,:2])
print('Y = \n',PS_subset_2sec.Y[:2,:2])
print('Z = \n',PS_subset_2sec.Z[:2,:2])
print('x[:5] = \n', PS_subset_2sec.x[:5])
print('y[:5] = \n', PS_subset_2sec.y[:5])

## Saving a DEM for use in GeoClaw

In [ ]:
fname = 'PS_subset_2sec_sample.tt3'
PS_subset_2sec.write(fname, topo_type=3)

This file contains a header in the fist 6 lines, followed by one line for each row of the topo array.  

Here is the header for the file just created:

In [ ]:
with open('PS_subset_2sec_sample.tt3','r') as my_file:
    for i in range(6):
        line = my_file.readline()
        print(line.strip())

#### Read it back in and check the coordinates:

In [ ]:
topo = topotools.Topography()
topo.read(fname, topo_type=3)
print('X = \n',topo.X[:2,:2])
print('Y = \n',topo.Y[:2,:2])
print('Z = \n',topo.Z[:2,:2])

## Write in asc format with `llcorner` registration:

Alternatively we could write it as:

In [ ]:
fname = 'PS_subset_2sec_sample.tt3'
PS_subset_2sec.write(fname, topo_type=3, header_style='asc', 
                     grid_registration='llcorner')

`header_style='asc'` causes the labels to appear before the values (either way works when reading into GeoClaw).

`grid_registration='llcorner'` causes it to be written with a different header, and with the lower left corner shifted by 1/2 grid cell (at the current resolution).

#### Print the header:

In [ ]:
with open('PS_subset_2sec_sample.tt3','r') as my_file:
    for i in range(6):
        line = my_file.readline()
        print(line.strip())

#### Read it back in and check the coordinates:

In [ ]:
topo = topotools.Topography()
topo.read(fname, topo_type=3)
print('X = \n',topo.X[:2,:2])
print('Y = \n',topo.Y[:2,:2])
print('Z = \n',topo.Z[:2,:2])

Note that once read back in, the GeoClaw tool adjusts `X` and `Y` to again be the proper points.

## Alignment of data

The etopo1 data from thredds has data that lies exactly on integer values of longitude and latitude, with spacing of 1 arcminute in between.  For example:

In [ ]:
print('X = \n',etopo1_subset.X[:2,:2])
print('Y = \n',etopo1_subset.Y[:2,:2])
print('Z = \n',etopo1_subset.Z[:2,:2])

The coastal DEMs have the same alignment.  For example, the first $X$ value in the `PS_subset_2sec` topography is 1/3" away from longitude $-123$.  We didn't pick up that point itself because of the choice of cropping region and coarsening factor.  Be careful what you ask for!

In [ ]:
x00 = PS_subset_2sec.X[0,0]
offset = (x00 + 123) * 3*3600.
print('X[0,0] is %.8f and lies %.6f times 1/3 arcsecond away from -123' % (x00, offset))